# Mapping and Hot-Spotting Oil-Spill Risk


#### Natalie Mesplay (A59025205)      Lyric Greif (A59024988)      Vittoria Trupiano (A16621484) 

## 3. Research Question and Significance

This project employs machine learning and spatial statistic techniques on remotely sensed data to examine spatial clustering of oil spills in the Mediterranean Sea. Specifically, we investigate whether oil spills tend to concentrate in particular locations within the Mediterranean Sea, and whether any locations contain statistically significant hotspots. Understanding these spatial patterns is important for identifying high-risk areas within the Mediterranean, which can support the development of oil spill response strategies in vulnerable regions of the sea. 

The primary audience for this analysis is the European Maritime Safety Agency, which is responsible for preventing and responding to incidents of ocean pollution. Additionally, individual shipping agencies, refineries, and regional port authorities can benefit from heightened awareness of oil spill risk areas. 

Using machine learning to identify spatial clusters is a valuable tool for visualizing the concentration of oil spill incidents within a particular region. Additionally, analyzing hotspots in the context of oil spills is important because it reveals statistically significant clusters of pollution concentration, highlighting areas where regulatory compliance can be improved. These insights support the prevention, monitoring, and cleanup of oil spills by delivering insight into regions which require additional monitoring. This knowledge is important for developing actionable policy recommendations that protect marine ecosystems and regulate the transport of petroleum products. 

Our final analysis changed significantly compared to our project proposal. Our original project goal was to determine which spatial and/or temporal clusters of spills portend poor port-reception compliance. The reason for the divergence was the lack of available data on vessel identification (MMSI or AIS), which prevented us from linking oil spill incidents to specific vessels and ports. Due to the limited availability of vessel and port identification data, we pivoted to performing a broader spatial analysis of oil spill distribution and concentration patterns. 


## 4. Background and Literature

Oil spills have devastating effects on fish, marine mammals, birds, and invertebrate species (Vieties et al., 2004). The Mediterranean Sea has a high incidence of oil spills due to heavy maritime traffic and industrial activities on the coast, including large numbers of refineries and petrochemical facilities (European Environmental Agency, 2025). Around 35% of oil entering the Mediterranean Sea originates from shipping activities, while 45% comes from land-based activities (European Environmental Agency, 2025).

Machine learning tools are increasingly being employed to identify and classify oil spills. Due to their devastating impact and diverse chemical properties, effective treatment of oil spills depends not only on timely identification, but also accurate classification (Bui et al., 2024), prediction (Bisen et al., 2024), and mapping. There are two primary methods used for identification of oil spills: direct surveillance using marine vehicles and remote sensing techniques. Several novel approaches build on these techniques to improve identification and classification methods through data augmentation (Bui et al., 2024) and prediction using vessel movement data (Bisen et al., 2024). Analyzing oil spill incident data can also be used to determine a baseline for the improvement of prevention and response activities (Park et al., 2016).

Machine learning can also help identify where oil spills are naturally clustered within a marine region. Insight on spatial concentration of oil spills is complemented by hotspot analysis, which is used to determine the location of statistically significant clusters of environmental hazards. Our analysis takes this route, using a combination of machine learning and spatial statistics to reveal areas of the Mediterranean Sea where oil spills are concentrated. 

The references included in this section helped us situate our project in the context of existing research using machine learning and spatial statistical analysis. With these sources in mind, we were better able to understand possible directions for our analysis and decided to focus on determining whether oil spills concentrate randomly or display statistically significant patterns. 

References: 
1. Durango-Cordero, Juan, Mehdi Saqalli, Christophe Laplanche, Marine Locquet, and Arnaud Elger. 2018. “Spatial Analysis of Accidental Oil Spills Using Heterogeneous Data: A Case Study from the North-Eastern Ecuadorian Amazon.” Sustainability 10 (12): 4719. https://doi.org/10.3390/su10124719.
2. Park, Yong, Hamad Al-Qublan, EunSu Lee, and Gokhan Egilmez. 2016. “Interactive Spatiotemporal Analysis of Oil Spills Using Comap in North Dakota.” Informatics 3 (2): 4. https://doi.org/10.3390/informatics3020004.
3. Bayramov, Emil, Kada ,Martin, and Manfred and Buchroithner. 2018. “Monitoring Oil Spill Hotspots, Contamination Probability Modelling and Assessment of Coastal Impacts in the Caspian Sea Using SENTINEL-1, LANDSAT-8, RADARSAT, ENVISAT and ERS Satellite Sensors.” Journal of Operational Oceanography 11 (1): 27–43. https://doi.org/10.1080/1755876X.2018.1438343.
4. European Environment Agency. 2025. “Oil Spills in Europe’s Seas.” March 2, 2025. https://www.eea.europa.eu/en/european-zero-pollution-dashboards/indicators/oil-spills-in-europes-seas.


## 5. Python Packages

### --- Core System & Web Utilities ---

#### `import os` used to interact with the operating system, such as file paths
#### `import requests` send HTTP requests to fetch data from APIs

In [1]:
import os  
import requests 

### --- Data Handling ---

#### Used to load, clean, and manage tabular datasets (e.g. vessel, spill, and centroid data)

In [2]:
import pandas as pd  

### --- Geospatial Tools ---

#### `import geopanda` as gpds used for loading, transforming, and plotting geographic features (e.g., vessel positions, spill areas)
#### `import contextily as ctx` adds basemaps for geographic context; used in `query_to_gdf_vis` to add OpenStreetMap tiles
#### 

In [3]:
import geopandas as gpd  
import contextily as ctx  
from shapely.geometry import Point

### --- Visualization ---

#### Used extensively to visualize clustering results and hot spot maps


In [4]:
import matplotlib.pyplot as plt 

###  --- ArcGIS Analysis Tools ---

#### `import arcgis`, as it is the main ArcGIS package enabling spatial analysis and GIS platform access 
#### `from arcgis.gis import GIS` logs in to ArcGIS Online and manage hosted layers. 
#### `from arcgis import geometry` is used for geometric operations, such as buffering and spatial queries.
#### `from arcgis.features import GeoAccessor, GeoSeriesAccessor` converts GeoDataFrames to Spatially Enabled DataFrames and vice versa, which is critical for integration between GeoPandas and ArcGIS.
#### `from IPython.display import display` displays DataFrames and results clearly in cells.
#### `from arcgis.features import analysis` is used would be for general geoprocessing (e.g., buffer, merge).
#### `from arcgis.features import analyze_patterns` are used for Hot Spot Analysis (Gi*) on vessel spill area data.
#### `from arcgis.features import summarize_data` summarizes spatial statistics over regions

In [5]:
import arcgis  
from arcgis.gis import GIS  
from arcgis import geometry  
from arcgis.features import GeoAccessor, GeoSeriesAccessor 
from IPython.display import display  
from arcgis.features import analysis  
from arcgis.features import analyze_patterns 
from arcgis.features import summarize_data

### --- Machine Learning (for Clustering) ---

#### `from sklearn.cluster import DBSCAN`  is a density-based clustering algorithm utilized in this project to identify spill clusters
#### `from sklearn.preprocessing import StandardScaler` scales spatial coordinates for better clustering performance

In [6]:
from sklearn.cluster import DBSCAN  # Density-based clustering algorithm (used to identify spill clusters)
from sklearn.preprocessing import StandardScaler  # Scales spatial coordinates for better clustering performance

In [ ]:
gis = GIS(username = 'gpec447sp25_27')

## 6. Data Sources

The primary data source for this project comes from SkyTruth Cerulean’s API, which detects both acute and chronic oil pollution from vessels and offshore oil platforms. SkyTruth Cerulean derives oil slick data from the Sentinel-1 satellite mission of the European Space Agency. Sentinel scenes are filtered to exclude land, then Skytruth applies its machine learning algorithm to determine which pixels contain oil. The model accomplishes this by pulling satellite images from the server, segmenting pixels to identify those containing oil, and creating a vector outline to represent the oil slick footprint. Finally, an algorithm is used to link oil slicks to a source: either broadcasting vessels, oil infrastructure, or dark vessels. 

Our data selection originally included the Maritime Mobile Service Identity (MMSI) number, which we planned to use to link spills to vessels and/or ports. However, the MMSI is unavailable through the Cerulean API. Therefore, we only used data made available through the API, which included oil slick geometry, slick ID, the timestamp of the oil slick, and the cause of the slick. Without vessel identification, we were unable to link spills with vessels, and subsequently to the ports to which the vessels travel. Future analysis would benefit from data on the origin and destinations of vessels involved with oil spills in order to better understand where vessels with poor environmental compliance call. 

## 7. Data Cleaning

The data cleaning process for this project consists of querying the Cerulean API to retrieve oil slick data in the western and eastern portions of the Mediterranean Sea. Query results were filtered to include oil slick detections with a machine confidence level over 90%, with dates filtered between Jan. 1st, 2020 and Jan. 1st, 2025. The raw data was converted to a GeoDataFrame and reprojected to EPSG: 3035 (the projected CRS for Europe).

The western and eastern segments of the Mediterranean Sea were joined and centroids were calculated for each oil slick geometry, then reprojected back into latitude and longitude. Finally, the coordinates of each point were extracted. These steps allowed for the visualization of oil slick geometries, and for clustering using DBSCAN. 

## 8. Descriptive Statistics for Data

Local spatial autocorrelation is detected by our hotspot analysis, so was not performed separately as a descriptive statistics. Similarly random point patterns were not analyzed as our project centers around the potential non-random patterns present in spills in the Mediterranean. 
Summary statistics were generated for the oil spill centroid dataset, showing the total number of points as 7161, the spatial mean of the dataset at (17.498106, 35.951041), and a standard distance of 5.037642. A map was additionally generated to visualize these summary statistics below.

## 9. Analysis

### Map Prep and Projecting

In [ ]:
def query_to_gdf_vis(data):
    gdf = gpd.GeoDataFrame.from_features(data["features"])
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs(epsg=3857)
    ax = gdf.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
    ctx.add_basemap(
        ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=gdf.crs.to_string()
    )
    plt.show()
    return gdf

def query_to_gdf_vis_col(data):
    gdf = gpd.GeoDataFrame.from_features(data["features"])
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs(epsg=3857)
    ax = gdf.plot(
        figsize=(10, 10),
        alpha=0.6,
        edgecolor="k",
        column='cluster',       # color by cluster
        cmap='tab10',           # categorical color map
        legend=True
    )
    plt.show()
    return gdf

In [ ]:
example_aoi_name = (
    "https://api.cerulean.skytruth.org/collections/public.aoi/items"  # This is the endpoint for all AOI data
    "?filter=LOWER(name) LIKE '%medit%'"  # filter by name (helps to search for what aois we can look at)
)

data = requests.get(example_aoi_name).json()
print("Number of results:", len(data["features"]))
print(data["features"][1]["properties"])

aoi_id = data["features"][0]["properties"]["id"]
aoi_id2 = data["features"][1]["properties"]["id"]

In [ ]:
def get_slicks_by_aoi(aoi_id):
    '''takes an aoi_id input and queries it with our specified characteristics'''
    url = (
        "https://api.cerulean.skytruth.org/collections/public.get_slicks_by_aoi/items"
        "?limit=9999" # limiting to 100 for now while testing (default if not specific is 10, max is 9999)
        "&sortby=-machine_confidence" # sort by machine confidence (how confident it is that its an actual spill)
        "&datetime=2020-01-01T00:00:00Z/2021-01-01T00:00:00Z"  # Limit results to a specific date range (jan 01 2020 to jan 01 2021)
        "&filter=machine_confidence GTE 0.90"  # Sort by machine confidence in descending order, only return spills with detection confidence greater than or equal to .90
        f"&aoi_id={aoi_id}" # by specific area of interest id, for us it's mediterranean west basin and med east basin : f is used bc API queries read in weird sometimes, this helps it read it the way we want
    )
    data = requests.get(url).json()
    # print("Response for AOI:", aoi_id)
    # print(data)  # add these two lines to debug if needed
    return query_to_gdf_vis(data)

# query both AOIs and combine the gdfs
gdf1 = get_slicks_by_aoi(aoi_id)
gdf2 = get_slicks_by_aoi(aoi_id2)


In [ ]:
def get_slicks_by_aoi(aoi_id):
    '''takes an aoi_id input and queries it with our specified characteristics'''
    url = (
        "https://api.cerulean.skytruth.org/collections/public.get_slicks_by_aoi/items"
        "?limit=9999" # limiting to 100 for now while testing (default if not specific is 10, max is 9999)
        "&sortby=-machine_confidence" # sort by machine confidence (how confident it is that its an actual spill)
        "&datetime=2021-01-01T00:00:00Z/2022-01-01T00:00:00Z"  # Limit results to a specific date range (jan 01 2021 to jan 01 2022)
        "&filter=machine_confidence GTE 0.90"  # Sort by machine confidence in descending order, only return spills with detection confidence greater than or equal to .90
        f"&aoi_id={aoi_id}" # by specific area of interest id, for us it's mediterranean west basin and med east basin : f is used bc API queries read in weird sometimes, this helps it read it the way we want
    )
    data = requests.get(url).json()
    # print("Response for AOI:", aoi_id)
    # print(data)  # add these two lines to debug if needed
    return query_to_gdf_vis(data)

# query both AOIs and combine the gdfs
gdf3 = get_slicks_by_aoi(aoi_id)
gdf4 = get_slicks_by_aoi(aoi_id2)


def get_slicks_by_aoi(aoi_id):
    '''takes an aoi_id input and queries it with our specified characteristics'''
    url = (
        "https://api.cerulean.skytruth.org/collections/public.get_slicks_by_aoi/items"
        "?limit=9999" # limiting to 100 for now while testing (default if not specific is 10, max is 9999)
        "&sortby=-machine_confidence" # sort by machine confidence (how confident it is that its an actual spill)
        "&datetime=2022-01-01T00:00:00Z/2023-01-01T00:00:00Z"  # Limit results to a specific date range (jan 01 2022 to jan 01 2023)
        "&filter=machine_confidence GTE 0.90"  # Sort by machine confidence in descending order, only return spills with detection confidence greater than or equal to .90
        f"&aoi_id={aoi_id}" # by specific area of interest id, for us it's mediterranean west basin and med east basin : f is used bc API queries read in weird sometimes, this helps it read it the way we want
    )
    data = requests.get(url).json()
    # print("Response for AOI:", aoi_id)
    # print(data)  # add these two lines to debug if needed
    return query_to_gdf_vis(data)

# query both AOIs and combine the gdfs
gdf5 = get_slicks_by_aoi(aoi_id)
gdf6 = get_slicks_by_aoi(aoi_id2)


In [ ]:
def get_slicks_by_aoi(aoi_id):
    '''takes an aoi_id input and queries it with our specified characteristics'''
    url = (
        "https://api.cerulean.skytruth.org/collections/public.get_slicks_by_aoi/items"
        "?limit=9999" # limiting to 100 for now while testing (default if not specific is 10, max is 9999)
        "&sortby=-machine_confidence" # sort by machine confidence (how confident it is that its an actual spill)
        "&datetime=2023-01-01T00:00:00Z/2024-01-01T00:00:00Z"  # Limit results to a specific date range (jan 01 2023 to jan 01 2024)
        "&filter=machine_confidence GTE 0.90"  # Sort by machine confidence in descending order, only return spills with detection confidence greater than or equal to .90
        f"&aoi_id={aoi_id}" # by specific area of interest id, for us it's mediterranean west basin and med east basin : f is used bc API queries read in weird sometimes, this helps it read it the way we want
    )
    data = requests.get(url).json()
    # print("Response for AOI:", aoi_id)
    # print(data)  # add these two lines to debug if needed
    return query_to_gdf_vis(data)

# query both AOIs and combine the gdfs
gdf7 = get_slicks_by_aoi(aoi_id)
gdf8 = get_slicks_by_aoi(aoi_id2)


In [ ]:
def get_slicks_by_aoi(aoi_id):
    '''takes an aoi_id input and queries it with our specified characteristics'''
    url = (
        "https://api.cerulean.skytruth.org/collections/public.get_slicks_by_aoi/items"
        "?limit=9999" # limiting to 100 for now while testing (default if not specific is 10, max is 9999)
        "&sortby=-machine_confidence" # sort by machine confidence (how confident it is that its an actual spill)
        "&datetime=2024-01-01T00:00:00Z/2025-01-01T00:00:00Z"  # Limit results to a specific date range (jan 01 2024 to jan 01 2025)
        "&filter=machine_confidence GTE 0.90"  # Sort by machine confidence in descending order, only return spills with detection confidence greater than or equal to .90
        f"&aoi_id={aoi_id}" # by specific area of interest id, for us it's mediterranean west basin and med east basin : f is used bc API queries read in weird sometimes, this helps it read it the way we want
    )
    data = requests.get(url).json()
    # print("Response for AOI:", aoi_id)
    # print(data)  # add these two lines to debug if needed
    return query_to_gdf_vis(data)

# query both AOIs and combine the gdfs
gdf9 = get_slicks_by_aoi(aoi_id)
gdf10 = get_slicks_by_aoi(aoi_id2)

In [ ]:
gdf_combined20 = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))
gdf_combined21 = gpd.GeoDataFrame(pd.concat([gdf3, gdf4], ignore_index=True))
gdf_combined22 = gpd.GeoDataFrame(pd.concat([gdf5, gdf6], ignore_index=True))
gdf_combined23 = gpd.GeoDataFrame(pd.concat([gdf7, gdf8], ignore_index=True))
gdf_combined24 = gpd.GeoDataFrame(pd.concat([gdf9, gdf10], ignore_index=True))

In [ ]:
gdf_final = gpd.GeoDataFrame(pd.concat(
    [gdf_combined20, gdf_combined21, gdf_combined22, gdf_combined23, gdf_combined24],
    ignore_index=True
))
gdf_final.crs

In [ ]:
gdf_final.head()
# gdf_combined.shape

In [ ]:
gdf_projected = gdf_final.to_crs(epsg=32633)
gdf_centroids = gdf_projected.copy()
gdf_centroids['geometry'] = gdf_projected['geometry'].centroid

In [ ]:
#gdf_centroids = gdf_centroids.to_crs(gdf_final.crs)

# Step 3: Extract centroid coordinates into new columns
gdf_centroids['longitude'] = gdf_centroids.geometry.x
gdf_centroids['latitude'] = gdf_centroids.geometry.y

# Step 4: Drop the geometry column if you only want CSV with coordinates
teamoilspillcentroids_csv = gdf_centroids.drop(columns='geometry')

# Step 5: Save to CSV
teamoilspillcentroids_csv.to_csv('teamoilspillcentroids.csv', index=False)

In [ ]:
#gdf_final['geometry'] = gdf_final['geometry'].apply(lambda geom: geom.wkt)

# Save to CSV
gdf_final.to_csv("teamoilspill.csv", index=False)

### Clustering

To investigate patterns in oil spill occurrences throughout the Mediterranean, we conducted both clustering and spatial statistical analyses on point data representing spill centroids. Our core analytical approach began with Density-Based Spatial Clustering of Applications with Noise (DBSCAN), a non-parametric clustering algorithm ideal for identifying spatial clusters of arbitrary shape and size. DBSCAN was chosen over other clustering techniques because of its effectiveness in handling large datasets with noise — a critical factor given the volume and variability of the spill data. Furthermore, while HDBSCAN could have been considered for its hierarchical capabilities, we prioritized DBSCAN for the greater manual control it provides in tuning minimum sample sizes and spatial radii, which allowed us to explore the clustering behavior at different density thresholds.


In [ ]:
#extract coordinates
coords = gdf_centroids[['longitude', 'latitude']].values


In [ ]:
# standardize coordinates
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)


In [ ]:
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

In [ ]:
# apply dbscan
db = DBSCAN(eps=0.08, min_samples=15)  # this looks the best
labels = db.fit_predict(coords_scaled)

In [ ]:
# add cluster labels back to GeoDataFrame
gdf_centroids['cluster'] = labels

In [ ]:
# plot the clusters
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(
    gdf_centroids['longitude'],
    gdf_centroids['latitude'],
    c=gdf_centroids['cluster'],
    cmap='tab10',
    s=10,
    alpha=0.6
)
plt.title('DBSCAN Clustering of Oil Slicks')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(scatter, label='Cluster Label')
plt.show()

In [ ]:
ax = gdf_centroids.plot(
        column='cluster',       # Color by this column
        cmap='tab10',           # Use a categorical color map
        figsize=(10, 10),
        edgecolor='black',
        linewidth=0.5,
        alpha=0.6
    )
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=gdf_centroids.crs)
ax.set_title("Spill Cluster Groups in the Mediterranean Sea")
ax.set_axis_off()
plt.show()

### k-NN Graph

To select the optimal spatial radius parameter (epsilon), we generated a k-nearest neighbors (k-NN) distance graph, with _k=10_. This graph plots the distances to each point’s 10th nearest neighbor in ascending order. The resulting curve revealed a distinct elbow at approximately 0.08, which in our projected coordinate reference system (EPSG:3035) translates to roughly 8 kilometers. The elbow marks the point where the marginal gain in neighbor inclusion plateaus, suggesting it as a natural threshold for the radius of influence around each spill. Based on this, we defined epsilon = 0.08 and set the minimum sample size to 15, which means that for any oil spill to be considered part of a cluster, it must have at least 14 other nearby spills within an 8 km radius. These parameters yielded a robust and interpretable clustering pattern, revealing three distinct spatial groupings across the Mediterranean region.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 10 nearest neighbors 
neighbors = NearestNeighbors(n_neighbors=10)
neighbors_fit = neighbors.fit(coords_scaled)
distances, _ = neighbors_fit.kneighbors(coords_scaled)

In [ ]:
# Sort distances to visualize the elbow
distances = np.sort(distances[:, -1])  # Use the 10th nearest neighbor distance
plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title('k-NN Distance Plot for Choosing eps')
plt.xlabel('Points sorted by distance to 10th nearest neighbor')
plt.ylabel('Distance')
plt.grid(True)
plt.show()

### Hot Spot Analysis

Beyond DBSCAN, we conducted a Hot Spot Analysis using the `find_hot_spots` function from the ArcGIS `analyze_patterns` module. This tool implements the Getis-Ord Gi* statistic to identify statistically significant spatial clusters of high (hot spots) and low (cold spots) values, using the density of spill points across a hexagonal grid overlay. Each hexagon's significance is measured via z-scores and p-values, which test the null hypothesis that the observed pattern could be due to random spatial distribution. Hex bins shaded in dark red represent hot spots with high spill density and strong statistical confidence (p < 0.01), suggesting consistent or anomalous activity in those areas. In contrast, dark blue bins identify cold spots with unusually low spill density. This method enriched our understanding of spatial intensity by layering statistical significance over raw point density, highlighting where policy or monitoring efforts may be especially critical.

In [ ]:
gdf_centroids.head()

In [ ]:
#Truncating column names to less than 10, and if they are duplicates than add 1,2,3...etc.
def truncate_column_names_safe(gdf, max_length=10):
    existing = set()
    new_columns = {}

    for col in gdf.columns:
        if len(col) <= max_length:
            new_col = col
        else:
            base = col[:max_length]
            new_col = base
            i = 1
            while new_col in existing or new_col in new_columns.values():
                suffix = f"_{i}"
                new_col = (base[:max_length - len(suffix)] + suffix)
                i += 1
        new_columns[col] = new_col
        existing.add(new_col)

    return gdf.rename(columns=new_columns)

# Use it
gdf_short = truncate_column_names_safe(gdf_centroids)



In [ ]:
gdf_short.head()

In [ ]:
sdf = GeoAccessor.from_geodataframe(gdf_short)
gdf_short.to_file("oil_spill_points.shp") 

# this produces a few errors on unprocessable lat/long coords - these are ignored

In [ ]:
# This cell uploads our hot spot to ArcGIS - don't run again bc the output_name will already exist and give you an error
#spills_fl = gis.content.get('48b841b969da4129be70d0cd9998fdee') # this is our centroid data uploaded as a FeatureLayer to ArcGIS Online


#spill_hot_spots = arcgis.features.analysis.find_hot_spots(spills_fl,
#                                     output_name='oil_spills_hotspots',
#                                     shape_type='hexagon', gis=gis)

spill_hot_spots = gis.content.get('b703defbd8fe432095ebdba924dc0a33') # this is just calling the hotspot layer we made in AGOL with the above code

# The error produced on CRS is ignored - the projected CRS is correct and the same

In [ ]:
spill_hot_spots

In [ ]:
m4 = gis.map('Mediterranean Sea')
m4.content.add(spill_hot_spots)
m4.legend.enabled = True
m4

### Spatial Mean and Standard Distance

To complement these clustering and hot spot techniques, we also calculated two spatial statistics: the spatial mean (a point representing the average location of all spill events) and standard distance (analogous to standard deviation, but in two-dimensional space). The spatial mean provided a central tendency for spill locations, while the standard distance visualized the overall geographic spread. These were plotted alongside our DBSCAN clusters and hot spot bins on basemaps from _OpenStreetMap_, which added interpretability by showing marine geography and nearby coastlines.

In [ ]:
mean_x = gdf_centroids.geometry.x.mean()
mean_y = gdf_centroids.geometry.y.mean()
spatial_mean = Point(mean_x, mean_y)

# Standard distance (similar to std dev but in space)
dists = np.sqrt((gdf_centroids.geometry.x - mean_x)**2 + (gdf_centroids.geometry.y - mean_y)**2)
standard_distance = dists.std()

fig, ax = plt.subplots(figsize=(10, 10))
gdf_centroids.plot(ax=ax, alpha=0.6, edgecolor='k')
gpd.GeoSeries([spatial_mean]).plot(ax=ax, color='red', marker='x', label='Spatial Mean')
circle = plt.Circle((mean_x, mean_y), standard_distance, color='orange', fill=False, label='Std Distance')
ax.add_patch(circle)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=gdf_centroids.crs)
ax.legend()
ax.set_title('Oil Spills: Spatial Mean and Standard Distance')

In [ ]:
print("Number of points:", len(gdf_centroids))
print("Spatial Mean:", spatial_mean)
print("Standard Distance:", standard_distance)

Collectively, these analyses offered a multi-layered view of spill dynamics — from raw clustering to statistical validation and spatial distribution. The combination of unsupervised machine learning, geostatistics, and visualization helped uncover not only where spills are happening, but also how often, how tightly grouped, and how likely they are to be non-random. This holistic approach lays the groundwork for predictive monitoring or further anomaly detection of vessels responsible for pollution.

## 10. Summary of Results

From this analysis of SkyTruth’s Cerulean data on satellite-detected oil spills, multiple statistically significant hot and cold spots for spills were detected in the Mediterranean Sea. The hotspots are concentrated around coastal areas, while the cold spots are generally found in areas further from shore. This was somewhat counterintuitive, as originally we believed areas further from shore would be more conducive to discreet intentional dumping. The hotspots for spills are in the northeastern Levantine Sea, concentrated north of Cyprus along the coast of Turkey from the port of Antalya, to the ports of Mersin and Iskenderun, and down the western coasts of Syria and Lebanon around the ports of Latakia and Tartus, as well as the port of Tripoli. Other hotspots include the eastern coast of Tunisia, around the ports of Sousse, Sfax, and Gabès. 

The southern and southwestern coast of Sicily is an additional hotspot, around Porto Empodocles, as well as the eastern coast of Algeria around the ports of Béjaïa, Skikda, and Annaba. Cold spots are concentrated further out in the sea between the islands of Sardinia and Mallorca, and in a swath along the coast of Libya and further out into the Mediterranean. The majority of these ports are cargo ports, and further research into this potential correlation is warranted. These findings nevertheless allow coastal states and Regional Fisheries Management Organizations (RFMOs) active in the area, like the General Fisheries Commission for the Mediterranean (GFCM) to better allocate their limited enforcement resources.


## 11. Discussion

These findings dovetail with the earlier literature review, suggesting that a decent portion of oil spills in the Mediterranean stem from industrial activities. It adds on to existing literature by introducing a consideration of major hotspots and examining nearby ports, and specifically by narrowing its focus to a specific geographic region with a history of spills. These results are important, especially to motivate further research on the topic regarding potential correlations between ports and spills in the Mediterranean.

An important tradeoff to mention would be the clustering technique, using DBSCAN. First, as opposed to using HDBSCAN, it allowed us to maintain control over the minimum samples, with the consequence of not utilizing the sophistication of the latter method. Regarding the epsilon value, depending on the accuracy or strategy, could create false clusters. 

Also, the data we used was based on Sentinel-1, with a temporal resolution of 6 days. Therefore, strong currents present in the Mediterranean have the potential to alter oil spill data. Some areas are relatively still, and others have more active currents, so our hotspots should be examined more closely in areas known to have stronger currents.

Hot spot analysis grid shapes pose an excellent strategy to view clusters within the grids. As opposed to squares, it eliminates the edge effect bias. Hexagons create a more balanced neighbor relationship, but at the cost of a more granular interpretation. 


## 12. Conclusion and Future Work

While we were unable to completely answer our initial research question of whether spatial and/or temporal clusters of spills portend poor port-reception compliance due to limitations within our dataset, we successfully explored spatial relationships within oil spill incident data in the Mediterranean Sea using machine learning techniques and through the detection of local spatial autocorrelation. Specifically, we revealed several clusters of oil spill incidents in the Mediterranean using the DBSCAN clustering method; we extended this exploration of oil spill data by conducting hot spot analysis to detect statistically significant clusters. Our approach is highly flexible and can be applied to other marine areas outside of the Mediterranean to determine whether concentrations of oil spills are statistically significant in other regions. Additional data on vessel identification would assist with linking oil spills to specific ports; while this data is available through the Cerulean it is not currently available via the API. If and when vessel identification data is made available through the API by SkyTruth, further analysis can be performed to link individual oil spills to specific vessels and their ports of call. This would allow for a more robust analysis of port reception non-compliance, as well as strengthened evaluation of monitoring and enforcement activities. 

We expect the results of our analysis to be used by maritime safety, environmental regulatory agencies, and local Coast Guard services in the region. Specific organizations that could make use of your findings are the European Maritime Safety Agency, the European Environment Agency, and the Coast Guards of states with significant oil spill clustering within their exclusive economic zones. 
